In [2]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain


import os
import json
import traceback
import pandas as pd
from dotenv import load_dotenv
import PyPDF2

In [3]:
load_dotenv()

True

In [4]:
key=os.getenv("OPENAI_API_KEY")

In [5]:
print(key)

sk-proj-EWD9l0HpGqd1WWyFFB57T3BlbkFJTNSXqfmP8Atdxf9eR2HX


In [7]:
llm=ChatOpenAI(openai_api_key=key,model_name="gpt-3.5-turbo", temperature=0.5)

c:\Users\misja\OneDrive\Desktop\JAHNAVI\GenAI\MCQ_GenAI_Project\MCQ-Generator-LLM\env\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [8]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001C73A76EF10>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001C73A781100>, temperature=0.5, openai_api_key='sk-proj-EWD9l0HpGqd1WWyFFB57T3BlbkFJTNSXqfmP8Atdxf9eR2HX', openai_proxy='')

In [9]:
with open(r"C:\Users\misja\OneDrive\Desktop\JAHNAVI\GenAI\MCQ_GenAI_Project\MCQ-Generator-LLM\Response.json","r") as f:
    RESPONSE_JSON= json.load(f)

In [10]:
print(RESPONSE_JSON)

{'1': {'mcq': 'multiple choice question', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}, '2': {'mcq': 'multiple choice question', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}, '3': {'mcq': 'multiple choice question', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}}


In [30]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs

{RESPONSE_JSON}

"""

In [31]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "RESPONSE_JSON"],
    template=TEMPLATE
    )

In [32]:
quiz_generation_prompt

PromptTemplate(input_variables=['RESPONSE_JSON', 'number', 'subject', 'text', 'tone'], template='\nText:{text}\nYou are an expert MCQ maker. Given the above text, it is your job to create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. \nMake sure the questions are not repeated and check all the questions to be conforming the text as well.\nMake sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make {number} MCQs\n\n{RESPONSE_JSON}\n\n')

In [33]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [15]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [16]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [17]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [34]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "RESPONSE_JSON"],
                                        output_variables=["quiz", "review"], verbose=True)

In [19]:
file_path= r"C:\Users\misja\OneDrive\Desktop\JAHNAVI\GenAI\MCQ_GenAI_Project\MCQ-Generator-LLM\data.txt"

In [20]:
print(file_path)

C:\Users\misja\OneDrive\Desktop\JAHNAVI\GenAI\MCQ_GenAI_Project\MCQ-Generator-LLM\data.txt


In [21]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [22]:
print(TEXT)

In the realm of cutting-edge technologies, Artificial Intelligence (AI) has become a ubiquitous term. However, it encompasses various subfields that can sometimes be confusing. By understanding their unique characteristics and applications, we can gain a clearer perspective on the evolving landscape of AI.

Let’s start with the word of the year: generative AI.

Generative AI is an advanced branch of AI that utilizes machine learning techniques to generate new, original content such as images, text, audio, and video. Unlike traditional machine learning, which focuses on mapping input to output, generative models aim to produce novel and realistic outputs based on the patterns and information present in the training data. Maybe you’ve played with Dall-E or chat GPT 4, these are all examples of Generative AI.

Machine Learning (ML):

Machine learning is a subset of AI that focuses on the development of algorithms that enable systems to learn from and make predictions or decisions based on

In [25]:
TEXT
NUMBER=5 
SUBJECT="AI"
TONE="simple"
RESPONSE_JSON= RESPONSE_JSON

In [27]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [35]:
class SimpleTokenCounter:
    def __init__(self):
        self.total_tokens = 0

    def count_tokens(self, text):
        self.total_tokens += len(text.split())

    def get_total_tokens(self):
        return self.total_tokens

token_counter = SimpleTokenCounter()


response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "RESPONSE_JSON": json.dumps(RESPONSE_JSON)
        }
        )

token_counter.count_tokens(TEXT)
print(f"Total tokens used: {token_counter.get_total_tokens()}")




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:In the realm of cutting-edge technologies, Artificial Intelligence (AI) has become a ubiquitous term. However, it encompasses various subfields that can sometimes be confusing. By understanding their unique characteristics and applications, we can gain a clearer perspective on the evolving landscape of AI.

Let’s start with the word of the year: generative AI.

Generative AI is an advanced branch of AI that utilizes machine learning techniques to generate new, original content such as images, text, audio, and video. Unlike traditional machine learning, which focuses on mapping input to output, generative models aim to produce novel and realistic outputs based on the patterns and information present in the training data. Maybe you’ve played with Dall-E or chat GPT 4, these are all examples of Generative AI.

Machine Learning (ML):

Machine learning is a subset of AI that focuses o

In [36]:
response

{'text': 'In the realm of cutting-edge technologies, Artificial Intelligence (AI) has become a ubiquitous term. However, it encompasses various subfields that can sometimes be confusing. By understanding their unique characteristics and applications, we can gain a clearer perspective on the evolving landscape of AI.\n\nLet’s start with the word of the year: generative AI.\n\nGenerative AI is an advanced branch of AI that utilizes machine learning techniques to generate new, original content such as images, text, audio, and video. Unlike traditional machine learning, which focuses on mapping input to output, generative models aim to produce novel and realistic outputs based on the patterns and information present in the training data. Maybe you’ve played with Dall-E or chat GPT 4, these are all examples of Generative AI.\n\nMachine Learning (ML):\n\nMachine learning is a subset of AI that focuses on the development of algorithms that enable systems to learn from and make predictions or 

In [37]:
quiz=response.get("quiz")

In [41]:
quiz= json.loads(quiz)

In [42]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [45]:
quiz_table_data

[{'MCQ': 'What is the main focus of generative AI?',
  'Choices': 'a: Mapping input to output | b: Generating new content | c: Learning from experience | d: Creating intelligent machines',
  'Correct': 'b'},
 {'MCQ': 'Which field of AI specializes in developing algorithms that learn from data to make predictions or decisions?',
  'Choices': 'a: Artificial Intelligence | b: Machine Learning | c: Generative AI | d: Robotics',
  'Correct': 'b'},
 {'MCQ': 'What distinguishes generative AI from traditional machine learning?',
  'Choices': 'a: It focuses on rule-based programming | b: It generates novel and realistic content | c: It mimics human-like cognitive abilities | d: It powers personalized recommendations',
  'Correct': 'b'},
 {'MCQ': 'Which field of AI involves explicitly defining rules and relationships between inputs and outputs?',
  'Choices': 'a: Machine Learning | b: Generative AI | c: Artificial Intelligence | d: Robotics',
  'Correct': 'c'},
 {'MCQ': 'What is the primary appl

In [46]:
quiz=pd.DataFrame(quiz_table_data)

In [47]:
quiz

,MCQ,Choices,Correct
0,What is the main focus of generative AI?,a: Mapping input to output | b: Generating new...,b
1,Which field of AI specializes in developing al...,a: Artificial Intelligence | b: Machine Learni...,b
2,What distinguishes generative AI from traditio...,a: It focuses on rule-based programming | b: I...,b
3,Which field of AI involves explicitly defining...,a: Machine Learning | b: Generative AI | c: Ar...,c
4,What is the primary application of machine lea...,a: Image synthesis | b: Text generation | c: P...,c


In [48]:
quiz.to_csv("ai_quiz.csv",index=False)

In [50]:
from datetime import datetime

LOG_FILE=f"{datetime.now().strftime('%m_%d_%Y_%H_%M_%S')}.log"

In [51]:
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'08_29_2024_04_58_19'